# Modeling perturbations

The :mod:`wetlab` schema is designed to model complex :class:`wetlab.Experiment` of any :class:`wetlab.ExperimentType` that include perturbations in the experimental design.
Perturbations are intentional disruptions in biological systems, such as gene knockouts (:class:`wetlab.GeneticTreatment`), drug treatments (:class:`wetlab.CompoundTreatment`), or environmental shifts (:class:`wetlab.EnvironmentalTreatment`), to study their effects on molecular and cellular processes.
We refer to perturbations that are applied to a system in an :class:`wetlab.Experiment` as 'treatment'.
Such treatments sometimes also have known :class:`TreatmentTarget` which can be one or serveral :class:`bionty.Gene`, :class:`bionty.Pathway`, or :class:`bionty.Protein`.
While single treatments are common, it is also possible to combine several treatments into a :class:`CombinationTreatment`.

In [ ]:
#! pip install 'lamindb[bionty]' wetlab
!lamin init --storage modeling-perturbations --name modeling-perturbations --schema bionty,wetlab

In [ ]:
import bionty as bt
import wetlab as wl

Gefitinib is primarily a pharmaceutical treatment, specifically a targeted therapy used in cancer treatment, particularly for non-small cell lung cancer (NSCLC).
Let's assume that we want to better understand the effectiveness of gefitinib and EGFR/KRAS knockdown combination treatment for people that have a smoking history.
In our early experimental setup, we therefore subject mice to smoke.

In [ ]:
bt.settings.organism = "mouse"

## Genetic treatments

We create two :class:`wl.GeneticTreatment` records associatede with the corresponding :class:`wl.TreatmentTarget`.

In [ ]:
EGFR_kd = wl.GeneticTreatment(
    system="CRISPR Cas9",
    name="EGFR knockdown",
    sequence="AGCTGACCGTGA",
    on_target_score=85,
    off_target_score=15
)
EGFR_kd.save()

In [ ]:
EGFR_gene = bt.Gene.from_public(symbol="EGFR")
EGFR_gene.save()
EGFR_kd_target = wl.TreatmentTarget(name="EGFR knockdown")
EGFR_kd_target.save()
EGFR_kd_target.genes.add(EGFR_gene)

In [ ]:
EGFR_kd.targets.add(EGFR_kd_target)
EGFR_kd_target

In [ ]:
KRAS_kd = wl.GeneticTreatment(
    system="CRISPR Cas9",
    name="KRAS",
    sequence="TTGGTGGTGAACT",
    on_target_score=100,
    off_target_score=20
)
KRAS_kd.save()

In [ ]:
KRAS_gene = bt.Gene.from_public(symbol="KRAS")
KRAS_gene.save()
KRAS_kd_target = wl.TreatmentTarget(name="KRAS knockdown")
KRAS_kd_target.save()
KRAS_kd_target.genes.add(KRAS_gene)

In [ ]:
KRAS_kd.targets.add(KRAS_kd_target)
KRAS_kd_target

## Compound treatments

Next, we create a :class:`wl.CompoundTreatment` records including associated :class:`wl.TreatmentTarget`.

In [ ]:
gefitinib = compound_treatment = wl.CompoundTreatment(
       name="gefitinib",
       pubchem_id=123631,
       ontology_id="CHEBI:49668"
    )
gefitinib.save()

Gefitinib is a tyrosine kinase inhibitor (TKI) that specifically targets the epidermal growth factor receptor (EGFR) pathway and the EGFR protein.
Therefore, we can also define the :class:`wl.TreatmentTarget` and associate it with a :class:`bionty.Pathway` and :class:`bionty.Protein`.

In [ ]:
egfr_pathway = bt.Pathway.from_public(name="epidermal growth factor receptor activity")
egfr_pathway.save()
egfr_protein = bt.Protein.from_public(uniprotkb_id="Q5SVE7")
egfr_protein.save()
egfr_targets = wl.TreatmentTarget(name="EGFR treatment targets")
egfr_targets.save()

In [ ]:
egfr_targets.pathways.add(egfr_pathway)
egfr_targets.proteins.add(egfr_protein)
egfr_targets

In [ ]:
gefitinib.targets.add(egfr_targets)
gefitinib

## Environmental Treatment

In [ ]:
smoking = wl.EnvironmentalTreatment(name="Smoking Status Measurement:", ontology_id="EFO_0006527")
smoking.save()

## Combination Treatment

Now we can combine all individual treatments into a single combination treatment.

In [ ]:
combination_treatment = wl.CombinationTreatment(name="gefitinib and EGFR/KRAS knockdown combination treatment subject to smoking")
combination_treatment.save()

In [ ]:
combination_treatment.genetics.set([EGFR_kd, KRAS_kd])
combination_treatment.compounds.set([gefitinib])
combination_treatment.environmentals.set([smoking])

In [ ]:
combination_treatment